# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


### **COVID Dashboard**


# Análise Retrospectiva da COVID-19 em 2021 E 2022: Casos, Mortes e Vacinações
Este projeto apresenta uma análise abrangente da evolução da pandemia de COVID-19 no Brasil ao longo dos anos de 2021 e 2022. Utilizando dados de casos confirmados, mortes e vacinações, o objetivo é fornecer uma visão clara dos principais indicadores da pandemia durante aquele ano. Através de gráficos interativos e mapas de calor, os usuários poderão explorar:

* **Total de Casos Confirmados:** Visualização do número acumulado de casos de
COVID-19 confirmados durante o ano.
* **Total de Mortes:** Exibição do número total de mortes atribuídas à COVID-19, destacando a gravidade da pandemia.
* **Total de Pessoas Vacinadas:**  Número de pessoas que receberam pelo menos uma dose da vacina, refletindo os esforços de imunização.
* **Taxa de Vacinação:** Percentual da população que foi vacinada com pelo menos uma dose, demonstrando a cobertura vacinal.

# Análises e Visualizações
* **Gráficos de Linhas:** Tendências ao longo do tempo dos casos confirmados, mortes e vacinações.
* **Gráficos de Barras:** Distribuição das vacinações por estado, permitindo comparações entre diferentes áreas.
* **Mapas de Calor:** Visualização da densidade de casos confirmados e vacinações em todos os estados, facilitando a identificação de áreas com alta cobertura vacinal.
* **Gráficos de Dispersão:** Exploração da relação entre o número de casos confirmados e a taxa de vacinação.
* **Tabelas Resumo:** Visão geral de casos, mortes e vacinações por estado, proporcionando uma análise detalhada e comparativa.


### **TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/8e1f2cc8-501f-4c8c-ad66-75917d36eaa2)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/code/deniseschuartz/projeto-ebac-dashboard-covid)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pelo Ministério da Saúde - Vacinômetro COVID-19 ([link](https://infoms.saude.gov.br/extensions/SEIDIGI_DEMAS_Vacina_C19/SEIDIGI_DEMAS_Vacina_C19.html)).

# Vacinação por Estado

In [ ]:
import math
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [ ]:
file_path = 'vacinação por estado Janeiro 2021 a dexembro 2022.xlsx'  # substitua pelo caminho do seu arquivo Excel
df = pd.read_excel(file_path)

# Salvar como CSV
data = 'vacinas_estado.csv'  # substitua pelo caminho onde deseja salvar o CSV
df.to_csv(data, index=False)

In [ ]:
df.head()

,UF,Mês,Total de Doses Aplicadas,1ª Dose,2ª Dose,3ª Dose,Dose Reforço,1° Dose Reforço,2° Dose Reforço,3° Dose Reforço,Dose Adicional,Dose Única
0,AC,2021-01,11129,11104,21,0,1,0,1,0,2,0
1,AL,2021-01,38245,38159,74,0,11,0,0,0,1,0
2,AM,2021-01,92545,92383,133,6,20,0,1,1,0,1
3,AP,2021-01,11583,11551,25,0,6,0,0,0,1,0
4,BA,2021-01,205435,203817,1300,0,209,0,11,1,85,12


In [ ]:
estado_map = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MG': 'Minas Gerais',
    'MS': 'Mato Grosso do Sul',
    'MT': 'Mato Grosso',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'PR': 'Paraná',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'RS': 'Rio Grande do Sul',
    'SC': 'Santa Catarina',
    'SE': 'Sergipe',
    'SP': 'São Paulo',
    'TO': 'Tocantins'
}

# Substituir abreviações pelos nomes completos dos estados
df['UF'] = df['UF'].replace(estado_map)

In [ ]:
df.head()

,UF,Mês,Total de Doses Aplicadas,1ª Dose,2ª Dose,3ª Dose,Dose Reforço,1° Dose Reforço,2° Dose Reforço,3° Dose Reforço,Dose Adicional,Dose Única
0,Acre,2021-01,11129,11104,21,0,1,0,1,0,2,0
1,Alagoas,2021-01,38245,38159,74,0,11,0,0,0,1,0
2,Amazonas,2021-01,92545,92383,133,6,20,0,1,1,0,1
3,Amapá,2021-01,11583,11551,25,0,6,0,0,0,1,0
4,Bahia,2021-01,205435,203817,1300,0,209,0,11,1,85,12


Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
df.to_csv('./covid-vacinas.csv', sep=',', index=False)

# Agrupamento vacinação - Brasil

In [ ]:
df_cleaned = df.drop(columns=['UF'])

df_brasil_vacinas = df_cleaned.groupby('Mês').sum().reset_index()

df_brasil_vacinas.head()

,Mês,Total de Doses Aplicadas,1ª Dose,2ª Dose,3ª Dose,Dose Reforço,1° Dose Reforço,2° Dose Reforço,3° Dose Reforço,Dose Adicional,Dose Única
0,2021-01,2856314,2844144,8663,66,2204,65,98,17,796,261
1,2021-02,6953853,4529725,2421149,12,1818,55,56,14,729,295
2,2021-03,17340620,13884720,3451895,43,2400,110,85,34,482,851
3,2021-04,24075282,12559345,11512267,62,2375,174,200,23,374,462
4,2021-05,21936336,15940814,5992522,179,1608,105,213,26,320,549


Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
df_brasil_vacinas.to_csv('./covid-vacinas-brasil.csv', sep=',', index=False)

# Casos por Estado

In [ ]:

def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
    date_range_days: int = (end_date - start_date).days
    for lag in range(date_range_days):
        yield start_date + timedelta(lag)

In [ ]:

start_date = datetime(2021, 1, 1)
end_date = datetime(2022, 12, 31)

In [ ]:

cases = []
for date in date_range(start_date=start_date, end_date=end_date):
    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'
    case = pd.read_csv(data_source_url, sep=',')
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
    cases.append(case)

In [ ]:

cases = pd.concat(cases, ignore_index=True)


In [ ]:

states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}


In [ ]:

cases = cases.rename(columns={
    'Province_State': 'UF',
    'Country_Region': 'pais',
    'Date': 'data',
    'Confirmed': 'confirmados',
    'Deaths': 'mortes'
})

In [ ]:
cases['UF'] = cases['UF'].apply(lambda state: states_map.get(state, state))

In [ ]:
cases['Mês'] = cases['data'].apply(lambda date: date.strftime('%Y-%m'))

In [ ]:
grouped_cases = cases.groupby(['UF', 'Mês']).agg({
    'confirmados': 'sum',
    'mortes': 'sum'
}).reset_index()

In [ ]:
def calculate_monthly_diff(group):
    group = group.sort_values(by='Mês')
    group['confirmados_1m'] = group['confirmados'].shift(1)
    group['mortes_1m'] = group['mortes'].shift(1)

    group['confirmados_1m'] = group['confirmados'] - group['confirmados_1m']
    group['mortes_1m'] = group['mortes'] - group['mortes_1m']

    group['confirmados_1m'] = group['confirmados_1m'].clip(lower=0)
    group['mortes_1m'] = group['mortes_1m'].clip(lower=0)

    return group

grouped_cases = grouped_cases.groupby('UF').apply(calculate_monthly_diff)



In [ ]:
grouped_cases.head(20)

UF      Mês  confirmados  mortes  confirmados_1m  mortes_1m
UF                                                                    
Acre 0   Acre  2021-01      1384282   25860             NaN        NaN
     1   Acre  2021-02      1486249   26019        101967.0      159.0
     2   Acre  2021-03      1972692   35276        486443.0     9257.0
     3   Acre  2021-04      2232100   41847        259408.0     6571.0
     4   Acre  2021-05      2490708   49940        258608.0     8093.0
     5   Acre  2021-06      2528903   51418         38195.0     1478.0
     6   Acre  2021-07      2683315   55045        154412.0     3627.0
     7   Acre  2021-08      2713745   56036         30430.0      991.0
     8   Acre  2021-09      2637299   54671             0.0        0.0
     9   Acre  2021-10      2727664   57090         90365.0     2419.0
     10  Acre  2021-11      2643561   55358             0.0        0.0
     11  Acre  2021-12      2737496   57339         93935.0     1981.0
     12  Acre  2022-01      2841440   57525        103944.0      186.0
     13  Acre  2022-02      3149623   54024        308183.0        0.0
     14  Acre  2022-03      3818237   61600        668614.0     7576.0
     15  Acre  2022-04      3728361   59903             0.0        0.0
     16  Acre  2022-05      3874285   62062        145924.0     2159.0
     17  Acre  2022-06      3759183   60062             0.0        0.0
     18  Acre  2022-07      4174364   62284        415181.0     2222.0
     19  Acre  2022-08      4568096   62750        393732.0      466.0

In [ ]:
grouped_cases['confirmados_1m'] = grouped_cases['confirmados_1m'].astype('Int64')
grouped_cases['mortes_1m'] = grouped_cases['mortes_1m'].astype('Int64')


In [ ]:
grouped_cases = grouped_cases[['UF','Mês',  'confirmados', 'confirmados_1m', 'mortes', 'mortes_1m']]


In [ ]:
grouped_cases.head()

UF      Mês  confirmados  confirmados_1m  mortes  mortes_1m
UF                                                                   
Acre 0  Acre  2021-01      1384282            <NA>   25860       <NA>
     1  Acre  2021-02      1486249          101967   26019        159
     2  Acre  2021-03      1972692          486443   35276       9257
     3  Acre  2021-04      2232100          259408   41847       6571
     4  Acre  2021-05      2490708          258608   49940       8093

In [ ]:
grouped_cases.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 648 entries, ('Acre', 0) to ('Tocantins', 647)
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   UF              648 non-null    object
 1   Mês             648 non-null    object
 2   confirmados     648 non-null    int64 
 3   confirmados_1m  621 non-null    Int64 
 4   mortes          648 non-null    int64 
 5   mortes_1m       621 non-null    Int64 
dtypes: Int64(2), int64(2), object(2)
memory usage: 56.1+ KB


In [ ]:
grouped_cases['taxa_mortalidade'] = (grouped_cases['mortes'] / grouped_cases['confirmados']) * 100
grouped_cases['taxa_mortalidade'] = grouped_cases['taxa_mortalidade'].round(2)

In [ ]:
grouped_cases.head()

UF      Mês  confirmados  confirmados_1m  mortes  mortes_1m  \
UF                                                                      
Acre 0  Acre  2021-01      1384282            <NA>   25860       <NA>   
     1  Acre  2021-02      1486249          101967   26019        159   
     2  Acre  2021-03      1972692          486443   35276       9257   
     3  Acre  2021-04      2232100          259408   41847       6571   
     4  Acre  2021-05      2490708          258608   49940       8093   

        taxa_mortalidade  
UF                        
Acre 0              1.87  
     1              1.75  
     2              1.79  
     3              1.87  
     4              2.01

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
grouped_cases.to_csv('./covid-casos.csv', sep=',', index=False)

# Agrupamento dos casos - Brasil

In [ ]:
import pandas as pd

print("Colunas do DataFrame:", grouped_cases.columns)
print("Índice do DataFrame:", grouped_cases.index)


df_reset = grouped_cases.reset_index(drop=True)


df_br = df_reset.groupby('Mês').agg({
    'confirmados': 'sum',
    'confirmados_1m': 'sum',
    'mortes': 'sum',
    'mortes_1m': 'sum',
    'taxa_mortalidade': 'mean'
}).reset_index()

df_br.rename(columns={
    'Mês': 'Mês',
    'confirmados': 'confirmados_totais',
    'confirmados_1m': 'confirmados_1m_totais',
    'mortes': 'mortes_totais',
    'mortes_1m': 'mortes_1m_totais',
    'taxa_mortalidade': 'taxa_mortalidade_media'
}, inplace=True)

df_br.head()


Colunas do DataFrame: Index(['UF', 'Mês', 'confirmados', 'confirmados_1m', 'mortes', 'mortes_1m',
       'taxa_mortalidade'],
      dtype='object')
Índice do DataFrame: MultiIndex([(     'Acre',   0),
            (     'Acre',   1),
            (     'Acre',   2),
            (     'Acre',   3),
            (     'Acre',   4),
            (     'Acre',   5),
            (     'Acre',   6),
            (     'Acre',   7),
            (     'Acre',   8),
            (     'Acre',   9),
            ...
            ('Tocantins', 638),
            ('Tocantins', 639),
            ('Tocantins', 640),
            ('Tocantins', 641),
            ('Tocantins', 642),
            ('Tocantins', 643),
            ('Tocantins', 644),
            ('Tocantins', 645),
            ('Tocantins', 646),
            ('Tocantins', 647)],
           names=['UF', None], length=648)


,Mês,confirmados_totais,confirmados_1m_totais,mortes_totais,mortes_1m_totais,taxa_mortalidade_media
0,2021-01,261425401,0,6489123,0,2.250370
1,2021-02,276779258,15913029,6721484,278193,2.211852
2,2021-03,361463134,84683876,8839837,2118353,2.250370
3,2021-04,412942432,51479298,10968427,2128590,2.427778
4,2021-05,484932463,71990031,13514947,2546520,2.521481


Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
df_br.to_csv('./covid-casos-nivel-nacional.csv', sep=',', index=False)